# This notebook will be used for the capstone project

In [2]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


##### Define Foursquare Credentials and Version

In [4]:
CLIENT_ID = 'V53MHPBBL23EVUFTF31HTKJAFRJIK2QL1WJURTZ3ARRFF3KE' # Foursquare ID
CLIENT_SECRET = 'G5Z0ALDAAKEOPESPXKBZPXNB2TIEJKGWM33ZGSBCNW0VGW0T' # Foursquare Secret
VERSION = '20180604'
LIMIT = 100

In [17]:
#address = 'Степная 40, Таганрог, Россия'
#address = 'Россия, Таганрог, Греческая 104'
address = 'Russia, Taganrog, Grecheskaya 104'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

47.21724655 38.9282073366964


In [27]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            latitude, 
            longitude, 
            500, 
            LIMIT)

In [42]:
results = requests.get(url).json()
import json, codecs
with open('work_nbrs.json', 'wb') as f:
    json.dump(results, codecs.getwriter('utf-8')(f), ensure_ascii=False)

In [44]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = [
        'Neighborhood', 'Latitude', 'Longitude', 'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category']
    
    return(nearby_venues)
        

In [45]:
venues = getNearbyVenues(
    names=['Grecheskaya'],
    latitudes=[47.21724655],
    longitudes=[38.9282073366964],
    radius=1000)
venues

,Neighborhood,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Grecheskaya,47.217247,38.928207,Театр имени А. П. Чехова,47.216325,38.928217,Theater
1,Grecheskaya,47.217247,38.928207,Зерно,47.216890,38.927160,Coffee Shop
2,Grecheskaya,47.217247,38.928207,Три Орешка,47.215685,38.931146,Coffee Shop
3,Grecheskaya,47.217247,38.928207,Центральный парк им. Горького,47.219068,38.926230,Park
4,Grecheskaya,47.217247,38.928207,Мясник и бык,47.212617,38.933402,Theme Restaurant
5,Grecheskaya,47.217247,38.928207,Осака,47.215520,38.929749,Japanese Restaurant
6,Grecheskaya,47.217247,38.928207,Гимназия А.П.Чехова,47.215869,38.921656,Historic Site
7,Grecheskaya,47.217247,38.928207,Булкин,47.213681,38.922515,Coffee Shop
8,Grecheskaya,47.217247,38.928207,Moloko Bar,47.212979,38.926611,Hookah Bar
9,Grecheskaya,47.217247,38.928207,Baskin Robins,47.215292,38.929711,Ice Cream Shop
